In [52]:
import os, sys, json, numpy, requests, tqdm, itertools
from pandas import Series, DataFrame, Timestamp, Timedelta
from pandas import DatetimeIndex, MultiIndex, date_range
from pandas import read_csv, concat

DATA_FOLDER = "./dataset"
DATA_FILE = "german-power-baseload-electricity-m-futures"

URL_DA = "https://api.energy-charts.info/price"
URL_HP = "https://api.energy-charts.info/public_power"

COUNTRIES = ["DE", "FR"]
SUBTYPES_HP = ["wind_onshore", "solar"]

### <u><b>Phase 1 - Data fetch</b></u>

#### <b>1. Energy futures' daily history</b>

In [ ]:
raw_mf = read_csv(f"{DATA_FOLDER}/{DATA_FILE}.csv")
raw_mf.index = DatetimeIndex(raw_mf.pop("Date"))
raw_mf["raw_px_change"] = raw_mf.pop("Change %").str.strip("%").astype(float) / 100
raw_mf.index = raw_mf.index.rename("time")
raw_mf.columns = raw_mf.columns.str.lower()
raw_mf = raw_mf.sort_index()

date_start = raw_mf.index.min()
date_end = raw_mf.index.max()

#### <b>2. Day-ahead prices</b>

In [ ]:
raw_da = DataFrame()
payload = {"start": date_start.isoformat(), "end": date_end.isoformat()}
print(f"Fetching day-ahead prices for {len(COUNTRIES)} countries...")
iterator = tqdm.tqdm(COUNTRIES)
for country in iterator:
    iterator.set_description(f"Country = {country}...")
    if (country == "DE"): country = "DE-LU"
    raw_da_i = requests.get(URL_DA, params = {"bzn": country, **payload}).json()
    raw_da_i = DataFrame(raw_da_i["price"], index = raw_da_i["unix_seconds"])
    raw_da_i.index = DatetimeIndex(raw_da_i.index * 1e9, name = "time")
    raw_da_i.columns = [country]
    raw_da = concat([raw_da, raw_da_i], axis = "columns")

raw_da = raw_da.sort_index()

#### <b>3. Historical public-power output </b>

In [ ]:
raw_hp = DataFrame()
payload = {"start": date_start.isoformat(), "end": date_end.isoformat()}
print(f"Fetching historical power output for {len(SUBTYPES_HP) * len(COUNTRIES)} elements...")
iterator = tqdm.tqdm(itertools.product(SUBTYPES_HP, COUNTRIES))
for subtype, country in iterator:
    country = country.lower()
    iterator.set_description(f"Subtype = {subtype}, Country = {country}...")
    raw_hp_i = requests.get(URL_HP, params = {"country": country, **payload, "subtype": subtype}).json()
    raw_hp_i = DataFrame(raw_hp_i["production_types"][0]["data"], index = raw_hp_i["unix_seconds"])
    raw_hp_i.index = DatetimeIndex(raw_hp_i.index * 1e9, name = "time")
    raw_hp_i.columns = [(country, subtype)]
    raw_hp = concat([raw_hp, raw_hp_i], axis = "columns")

raw_hp = raw_hp.sort_index()
raw_hp.columns = MultiIndex.from_tuples(raw_hp.columns)
#raw_hp.index = raw_hp.index.floor("D")
#raw_hp = raw_hp.groupby("time").sum() # To daily data

Fetching day-ahead prices for 2 countries...


Country = FR...: 100%|██████████| 2/2 [00:10<00:00,  5.09s/it]


Fetching historical power output for 4 elements...


Subtype = solar, Country = fr...: : 4it [00:34,  8.64s/it]       


### <u><b>Phase 2 - Feature engineering</b></u>

#### <b>2/3. Front-month derived features</b>

In [89]:
fm = raw_mf.copy()
fm["price_change"] = fm["price"].diff()
fm["fm_d1"] = fm["price_change"].shift(1)

n_days_fm = 5
fm[f"fm_d{n_days_fm}"] = fm["price_change"].shift(n_days_fm)
fm[f"fm_trend_d{n_days_fm}"] = fm[f"fm_d{n_days_fm}"].rolling(n_days_fm).mean()

n_days_zs = 20
fm[f"fm_z_{n_days_zs}"] = fm["price"].shift(1) - fm["price"].rolling(n_days_zs).mean()
fm[f"fm_z_{n_days_zs}"] /= fm["price"].rolling(n_days_zs).std()

fm

,price,open,high,low,vol.,raw_px_change,price_change,fm_d1,fm_d5,fm_trend_d5,fm_z_20
time,,,,,,,,,,,
2023-01-03,142.38,142.38,142.38,142.38,NaN,-0.4341,NaN,NaN,NaN,NaN,NaN
2023-01-04,126.42,126.42,126.42,126.42,NaN,-0.1121,-15.96,NaN,NaN,NaN,NaN
2023-01-05,130.66,130.66,130.66,130.66,NaN,0.0335,4.24,-15.96,NaN,NaN,NaN
2023-01-06,124.81,124.81,124.81,124.81,NaN,-0.0448,-5.85,4.24,NaN,NaN,NaN
2023-01-09,121.01,121.01,121.01,121.01,NaN,-0.0304,-3.80,-5.85,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2025-11-05,107.37,107.37,107.37,107.37,NaN,-0.0612,-7.00,4.62,-0.81,0.684,2.539902
2025-11-06,103.43,103.43,103.43,103.43,NaN,-0.0367,-3.94,-7.00,-5.36,-0.328,1.601459
2025-11-07,103.15,103.15,103.15,103.15,NaN,-0.0027,-0.28,-3.94,15.24,2.686,1.068051


#### <b>4. Day-ahead (DA) price based features</b>

Note that day‑ahead auction results for delivery day T+1 are
known from ~12:57 CET on T (orderbook closes 12:00 CET).

So DA‑based and RES‑forecast features can legitimately explain
same‑day futures moves.

In [138]:
da = raw_da.copy()
ob_close_hour = 13
da = da.loc[da.index.hour == ob_close_hour]
da.index = da.index.floor(Timedelta(hours = 1))
da = da.groupby("time").first()
da.index = da.index.floor(Timedelta(days = 1))

da = da.rename_axis("country", axis = "columns")
da = da.stack("country").swaplevel().sort_index()
da = da.rename("da").to_frame().sort_index()

da["da_de_roll7"] = da.groupby("country")["da"].rolling(7).mean().reset_index(0, drop = True)
da["da_de_roll28"] = da.groupby("country")["da"].rolling(28).mean().reset_index(0, drop = True)
da["da_de_trend_xover"] = da["da_de_roll7"] - da["da_de_roll28"]
da["da_de_trend_xover_ramp"] = da["da_de_trend_xover"].diff()

for country in da.index.get_level_values(0).unique():
    print(f"Day-ahead price based features for {country}:")
    display(da.loc[country].dropna())

Day-ahead price based features for DE-LU:


,da,da_de_roll7,da_de_roll28,da_de_trend_xover,da_de_trend_xover_ramp
time,,,,,
2023-01-31,106.62,144.092857,129.713571,14.379286,-13.507143
2023-02-01,95.97,130.417143,130.276071,0.141071,-14.238214
2023-02-02,161.44,127.582857,130.647857,-3.065000,-3.206071
2023-02-03,97.91,115.641429,129.869286,-14.227857,-11.162857
2023-02-04,134.09,115.060000,131.480357,-16.420357,-2.192500
...,...,...,...,...,...
2025-11-06,78.51,67.800000,59.872857,7.927143,11.240714
2025-11-07,80.82,67.990000,60.027857,7.962143,0.035000
2025-11-08,86.33,74.112857,61.162857,12.950000,4.987857


Day-ahead price based features for FR:


,da,da_de_roll7,da_de_roll28,da_de_trend_xover,da_de_trend_xover_ramp
time,,,,,
2023-01-31,150.91,161.762857,140.738571,21.024286,-8.760357
2023-02-01,131.30,153.018571,141.675357,11.343214,-9.681071
2023-02-02,161.44,150.710000,142.047143,8.662857,-2.680357
2023-02-03,134.14,143.944286,142.222857,1.721429,-6.941429
2023-02-04,134.09,143.452857,143.754286,-0.301429,-2.022857
...,...,...,...,...,...
2025-11-06,76.85,19.552857,22.173929,-2.621071,6.716429
2025-11-07,45.00,23.838571,22.650714,1.187857,3.808929
2025-11-08,57.80,29.952857,24.402143,5.550714,4.362857


#### <b>5. Renewable-based features</b>

Compute ramps (how today differs from yesterday)
* wind_ramp = wind_onshore MWh(t) − wind_onshore MWh(t-1)
* solar_ramp = solar MWh(t) − solar MWh(t-1)
* total_ramp = total(t) - total(t-1) = solar_ramp + wind_ramp

(total is the sum of wind and solar)

In [163]:
hp = raw_hp.copy().swaplevel(axis = "columns")
hp = hp.rename_axis(["subtype", "country"], axis = "columns")
hp = hp.stack(hp.columns.names).rename("hp").reset_index()
hp["country"] = hp["country"].str.upper() # Make compatible with DA
hp.loc[hp["country"] == "DE", "country"] = "DE-LU"

# net daily power per country per subtype

hp["time"] = hp["time"].dt.floor("D")
# I guess each bar's value is the power output during such interval...
# So then the total daily power is the sum of all bars within such day
hp = hp.groupby(["country", "subtype", "time"]).sum()
hp = hp.sort_index().unstack("subtype")["hp"]

hp["wind_ramp"] = hp["wind_onshore"].diff()
hp["solar_ramp"] = hp["solar"].diff()
hp["total_ramp"] = hp["wind_ramp"] + hp["solar_ramp"]

for country in hp.index.get_level_values(0).unique():
    print(f"Day-ahead price based features for {country}:")
    display(hp.loc[country].dropna())

Day-ahead price based features for DE-LU:


C:\Users\GSL\AppData\Local\Temp\ipykernel_34980\3016986723.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  hp = hp.stack(hp.columns.names).rename("hp").reset_index()


subtype,solar,wind_onshore,wind_ramp,solar_ramp,total_ramp
time,,,,,
2023-01-04,-255975.4,-255975.4,-137665.6,-137665.6,-275331.2
2023-01-05,-151186.5,-151186.5,104788.9,104788.9,209577.8
2023-01-06,-166064.4,-166064.4,-14877.9,-14877.9,-29755.8
2023-01-07,-156322.4,-156322.4,9742.0,9742.0,19484.0
2023-01-08,-224999.0,-224999.0,-68676.6,-68676.6,-137353.2
...,...,...,...,...,...
2025-11-07,-83294.1,-83294.1,5021.3,5021.3,10042.6
2025-11-08,-84502.4,-84502.4,-1208.3,-1208.3,-2416.6
2025-11-09,-142585.5,-142585.5,-58083.1,-58083.1,-116166.2


Day-ahead price based features for FR:


subtype,solar,wind_onshore,wind_ramp,solar_ramp,total_ramp
time,,,,,
2023-01-03,-12368.0,-12368.0,-9985.3,-9985.3,-19970.6
2023-01-04,-30440.0,-30440.0,-18072.0,-18072.0,-36144.0
2023-01-05,-19172.0,-19172.0,11268.0,11268.0,22536.0
2023-01-06,-19548.0,-19548.0,-376.0,-376.0,-752.0
2023-01-07,-41822.0,-41822.0,-22274.0,-22274.0,-44548.0
...,...,...,...,...,...
2025-11-07,-58255.6,-58255.6,-14820.6,-14820.6,-29641.2
2025-11-08,-96225.8,-96225.8,-37970.2,-37970.2,-75940.4
2025-11-09,-92206.0,-92206.0,4019.8,4019.8,8039.6


#### <b>6. Calendar features</b>

In [ ]:
cal = fm.copy()
cal["day_of_week"] = cal.index.dayofweek
cal["day_of_month"] = cal.index.day
cal = cal.iloc[:, -2 :]

,day_of_week,day_of_month
time,,
2023-01-03,1,3
2023-01-04,2,4
2023-01-05,3,5
2023-01-06,4,6
2023-01-09,0,9
...,...,...
2025-11-05,2,5
2025-11-06,3,6
2025-11-07,4,7


### <b><u>All features</u></b>

In [205]:
main_da, main_hp = da.copy(), hp.copy()
main_da = main_da.unstack("country")
main_hp = main_hp.unstack("country")
main_da.columns = main_da.columns.map("_".join)
main_hp.columns = main_hp.columns.map("_".join)

main = fm.reset_index()
main = DataFrame.merge(main, cal.reset_index(), on = "time")
main = DataFrame.merge(main, main_da.reset_index(), on = "time")
main = DataFrame.merge(main, main_hp.reset_index(), on = "time")
main = main.set_index("time").sort_index().round(8)
main.to_csv(DATA_FOLDER + "/main_features.csv")
main

,price,open,high,low,vol.,raw_px_change,price_change,fm_d1,fm_d5,fm_trend_d5,...,solar_DE-LU,solar_FR,wind_onshore_DE-LU,wind_onshore_FR,wind_ramp_DE-LU,wind_ramp_FR,solar_ramp_DE-LU,solar_ramp_FR,total_ramp_DE-LU,total_ramp_FR
time,,,,,,,,,,,,,,,,,,,,,
2023-01-03,142.38,142.38,142.38,142.38,NaN,-0.4341,NaN,NaN,NaN,NaN,...,-118309.8,-12368.0,-118309.8,-12368.0,NaN,-9985.3,NaN,-9985.3,NaN,-19970.6
2023-01-04,126.42,126.42,126.42,126.42,NaN,-0.1121,-15.96,NaN,NaN,NaN,...,-255975.4,-30440.0,-255975.4,-30440.0,-137665.6,-18072.0,-137665.6,-18072.0,-275331.2,-36144.0
2023-01-05,130.66,130.66,130.66,130.66,NaN,0.0335,4.24,-15.96,NaN,NaN,...,-151186.5,-19172.0,-151186.5,-19172.0,104788.9,11268.0,104788.9,11268.0,209577.8,22536.0
2023-01-06,124.81,124.81,124.81,124.81,NaN,-0.0448,-5.85,4.24,NaN,NaN,...,-166064.4,-19548.0,-166064.4,-19548.0,-14877.9,-376.0,-14877.9,-376.0,-29755.8,-752.0
2023-01-09,121.01,121.01,121.01,121.01,NaN,-0.0304,-3.80,-5.85,NaN,NaN,...,-166702.0,-20615.0,-166702.0,-20615.0,58297.0,12178.0,58297.0,12178.0,116594.0,24356.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-04,114.37,114.37,114.37,114.37,NaN,0.0421,4.62,10.11,5.22,0.658,...,-138741.1,-106646.1,-138741.1,-106646.1,20571.3,-18890.7,20571.3,-18890.7,41142.6,-37781.4
2025-11-05,107.37,107.37,107.37,107.37,NaN,-0.0612,-7.00,4.62,-0.81,0.684,...,-135028.8,-71947.5,-135028.8,-71947.5,3712.3,34698.6,3712.3,34698.6,7424.6,69397.2
2025-11-06,103.43,103.43,103.43,103.43,NaN,-0.0367,-3.94,-7.00,-5.36,-0.328,...,-88315.4,-43435.0,-88315.4,-43435.0,46713.4,28512.5,46713.4,28512.5,93426.8,57025.0


### <u><b>Phase 3 - Feature selection</b></u>

Goal: find a subset with predictive power for price_change.
Implement these three complementary methods in a jupyter notebook:

1. Filtering: Rank by Spearman correlation and mutual information
(sklearn.feature_selection.mutual_info_regression).
2. Embedded: LassoCV (L1) on standardized features. Report non‑zero
coefficients and their magnitudes.
3. Model‑agnostic: Permutation importance using a small RandomForestRegressor. Report top contributors.

In [209]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
main = read_csv(DATA_FOLDER + "/main_features.csv", index_col = "time", parse_dates = True)

main["target"] = main["price_change"].shift(-1)
X = main.drop(columns = ["vol.", "target", "price"]).dropna()
y = main["target"].dropna()

common_index = X.index.intersection(y.index)
X = X.loc[common_index]
y = y.loc[common_index]

# Filtering: Rank by Spearman correlation and mutual information
res = Series(mutual_info_regression(X, y), index = X.columns)
print("Top 10 Mutual information coefficients:")
print(res.sort_values(ascending = False).round(6).head(10))

Top 10 Mutual information coefficients:
day_of_month                    0.134332
da_de_roll7_FR                  0.081074
da_de_roll28_DE-LU              0.055308
open                            0.045324
low                             0.045159
high                            0.044983
da_de_trend_xover_ramp_DE-LU    0.038676
da_de_roll28_FR                 0.033426
solar_FR                        0.022873
wind_onshore_FR                 0.021829
dtype: float64
